In [1]:
%cd ..

import importlib
import json
import math
import os
import socket
import sys
import time


internal_path = os.path.abspath(os.path.join('.'))
sys.path.append(internal_path)
sys.path.append(internal_path + '/datasets')
sys.path.append(internal_path + '/utils')

import datetime
import uuid
from argparse import ArgumentParser

import setproctitle
import torch
import numpy as np
import pandas as pd 
import json
import wandb

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import AutoMinorLocator
from matplotlib.collections import LineCollection



/pub/hofmann-scratch/glanzillo/ded


In [2]:
os.environ['WANDB_API_KEY'] = 'c7314b16e17009f66f3043c0b7968e1142054123'
wandb.login()

api = wandb.Api()
runs = api.runs(path='continually/DataEfficientDistillation', filters={'tags': 'useful'})
print("Loaded", len(runs), "runs")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: glanzillo (continually). Use `wandb login --relogin` to force relogin


Loaded 10 runs


In [3]:
dir(runs)

['QUERY',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_include_sweeps',
 '_load_page',
 '_sweeps',
 'client',
 'convert_objects',
 'cursor',
 'entity',
 'filters',
 'index',
 'last_response',
 'length',
 'more',
 'next',
 'objects',
 'order',
 'per_page',
 'project',
 'update_variables',
 'variables']

In [4]:
dir(runs.objects[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_attrs',
 '_base_dir',
 '_entity',
 '_exec',
 '_files',
 '_full_history',
 '_include_sweeps',
 '_repr_html_',
 '_sampled_history',
 '_state',
 '_summary',
 'client',
 'create',
 'delete',
 'dir',
 'display',
 'entity',
 'file',
 'files',
 'history',
 'id',
 'json_config',
 'lastHistoryStep',
 'load',
 'log_artifact',
 'logged_artifacts',
 'name',
 'path',
 'project',
 'save',
 'scan_history',
 'snake_to_camel',
 'state',
 'storage_id',
 'summary',
 'sweep',
 'to_html',
 'update',
 'upload_file',
 'url',
 'use_artifact',
 'used_artifacts',
 'user',
 'username',
 'wait_until_finished']

In [8]:
config = json.loads(runs.objects[0].json_config)
config.keys()

dict_keys(['mse', 'sgd', 'seed', 'alpha', 'debug', 'h_dim', 'device', 'exp_id', 'max_lr', 'min_lr', 'compile', 'dataset', 'mlp_dim', 'n_heads', 'seq_len', 'exp_name', 'head_dim', 'log_path', 'n_layers', 'data_root', 'n_workers', 'use_flash', 'eval_every', 'vocab_size', 'decay_steps', 'logger_type', 'temperature', 'wandb_notes', 'project_path', 'grad_clip_norm', 'log_ckpt_every', 'max_eval_steps', 'eval_batch_size', 'log_grads_every', 'max_train_steps', 'test_batch_size', 'train_batch_size', 'log_metrics_every', 'relative_log_path', 'tokens_per_second', 'wandb_entity_name', 'log_terminal_every', 'wandb_project_name', 'log_alignments_every', 'log_activations_every', 'teacher_checkpoint_path', 'gradient_accumulation_steps'])

In [17]:
config['seed']['value']

55

In [10]:
h = runs.objects[0].history()
h.head()

,_step,config,_runtime,_timestamp,parameter count,parameter count without embedding,eval_batches,eval_tokens,eval_bytes,_train/tokens_seen,...,_eval/normalised_ppl_over_tokens,_eval/normalised_loss,_eval/normalised_ppl_over_gpuseconds,x_axes/gpu_seconds,_time/tokens_per_second,_eval/normalised_loss_over_gpuseconds,_eval/ppl_over_gpuseconds,_train/loss,_time/iterations_per_second,_eval/top10_acc
0,0,"{'size': 1390, '_latest_artifact_path': 'wandb...",9.283063,1.705768e+09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,9.298753,1.705768e+09,27549696.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,9.298922,1.705768e+09,NaN,10772480.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,46.049116,1.705768e+09,NaN,NaN,500.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,46.049463,1.705768e+09,NaN,NaN,NaN,4096000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
h.columns

Index(['_step', 'config', '_runtime', '_timestamp', 'parameter count',
       'parameter count without embedding', 'eval_batches', 'eval_tokens',
       'eval_bytes', '_train/tokens_seen', '_train/learning_rate_0',
       '_eval/total_loss', '_eval/top10_acc_over_gpuseconds', '_eval/ppl',
       '_eval/fa', '_eval/ppl_over_tokens',
       '_eval/normalised_loss_over_tokens', '_eval/top1_acc_over_tokens',
       'x_axes/n_tokens', '_time/forward', '_time/eval',
       '_eval/top1_acc_over_gpuseconds', '_eval/top10_acc_over_tokens',
       '_eval/cka', '_eval/loss', '_time/train_step', '_time/total_step',
       '_train/cka', '_time/backward', '_train/fa', '_eval/normalised_ppl',
       '_eval/top1_acc', '_time/load_batch',
       '_eval/normalised_ppl_over_tokens', '_eval/normalised_loss',
       '_eval/normalised_ppl_over_gpuseconds', 'x_axes/gpu_seconds',
       '_time/tokens_per_second', '_eval/normalised_loss_over_gpuseconds',
       '_eval/ppl_over_gpuseconds', '_train/loss',
     

In [25]:
list(h[6:]['_eval/cka'])

[0.40231186151504517,
 0.5032689571380615,
 0.5224665403366089,
 0.536484956741333,
 0.5498769283294678,
 0.5514689087867737,
 0.5602566003799438,
 0.5614172220230103,
 0.5586612820625305,
 0.5795760750770569,
 0.5739375352859497,
 0.5710100531578064,
 0.5867127776145935,
 0.5773130059242249,
 0.5850902199745178,
 0.591157078742981,
 0.592695415019989,
 0.5914744734764099,
 0.6026270389556885,
 0.5916302800178528,
 0.592689573764801,
 0.5996255874633789,
 0.6005229949951172,
 0.6010473966598511,
 0.6057960391044617,
 0.6048132181167603,
 0.6079581379890442,
 0.6118248701095581,
 0.6088635325431824,
 0.6044564843177795,
 0.6107466220855713,
 0.6121087670326233,
 0.623515784740448,
 0.6232402324676514,
 0.6092463135719299,
 0.6210176944732666,
 0.6167471408843994,
 0.6087719798088074,
 0.6146456599235535,
 0.6195009350776672,
 0.6198665499687195,
 0.616502046585083,
 0.6148177981376648,
 0.6190458536148071,
 0.6199896931648254,
 0.6132553815841675,
 0.6244933009147644,
 0.606181204319000

In [30]:

COLUMNS = ['alpha','seed','student_net','teacher_net','step',
           'tokens_seen','cka_val','cka_train','fa_val','fa_train',
           'val_acc_top1', 'val_acc_top10', 'loss_train', 'ppl_val']
res_df = pd.DataFrame(columns=COLUMNS)

for run in runs.objects:
    h = run.history()[6:] # first rows are for validation
    config = json.loads(run.json_config)
    total = len(h)
    new_runs = '_eval/cka' in h.columns
    sub_df = pd.DataFrame({"alpha":[config['alpha']['value']]*total,
                           "seed":[config['seed']['value']]*total,
                           #"student_net":[]*total, #TODO: add tag
                           #"teacher_net":[]*total,
                           "step":list(h['_step']),
                           "tokens_seen":list(h['_train/tokens_seen']),
                           "cka_val": list(h['_eval/cka']) if new_runs else [-1]*total ,
                           "cka_train":list(h['_train/cka']) if new_runs else [-1]*total,
                           "fa_val":list(h['_eval/fa']) if new_runs else [-1]*total,
                           "fa_train":list(h['_train/fa']) if new_runs else [-1]*total,
                           "val_acc_top1":list(h['_eval/top1_acc']),
                           "val_acc_top10":list(h['_eval/top1_acc']),
                           "loss_train":list(h['_train/loss']),
                           "ppl_val":list(h['_eval/ppl'])
                        })
    res_df = pd.concat([res_df,sub_df], ignore_index=True)


In [35]:
res_df.iloc[1000]

alpha                      1
seed                      55
student_net              NaN
teacher_net              NaN
step                    6600
tokens_seen      432603136.0
cka_val             0.679556
cka_train           0.712425
fa_val              0.004591
fa_train             0.00477
val_acc_top1        0.331598
val_acc_top10       0.331598
loss_train          3.467307
ppl_val             39.10051
Name: 1000, dtype: object